In [74]:
from pathlib import Path

# 1. Get data from the .chart file

In [75]:
# Let's start by opening a .chart file and reading it

# Define path to chart
d = str(Path().resolve().parent)
chartpath = d+'\\tensor-hero\Training Data\Audioslave - Exploder (Chezy)\\notes.chart'

# Read chart into array
with open(chartpath, 'r') as file:
    raw_chart = file.readlines()
file.close()

# Strip lines of \n character
for i in range(len(raw_chart)):
    raw_chart[i] = raw_chart[i].replace('\n', '')

In [76]:
# Create lists to hold sections of of the chart file
song = []
synctrack = []
expertsingle = []

# Parse chart file, populating lists
i = 0
for data in raw_chart:
    if data in ['[Song]', '[SyncTrack]', '[Events]', '[ExpertSingle]']:
        i += 1
    
    if data in ['{', '}', '[Song]', '[SyncTrack]', '[Events]', '[ExpertSingle]']:
        continue
    elif i == 1:
        song.append(data[2:])
    elif i == 2:
        synctrack.append(data[2:])
    elif i == 3:
        continue
    elif i == 4:
        expertsingle.append(data[2:])



In [77]:
# Parse the 'synctrack' section of the .chart file for timing information
time_signatures = {'tick' : [],
                   'TS' : []}
BPMs = {'tick' : [],
        'BPM' : []}

for event in synctrack:
    line = event.split(' ')
    if line[2] == 'TS':
        time_signatures['tick'].append(int(line[0]))
        time_signatures['TS'].append(int(line[3]))
    elif line[2] == 'R':
        raise NameError('Error - Resolution changes during this song')
    else:
        BPMs['tick'].append(int(line[0]))
        BPMs['BPM'].append(int(line[3]))

In [84]:
# Parse the 'expertsingle' section of the .chart file for note information
notes = {'tick' : [],        # What tick the note is at
         'N_S' : [],         # Whether it is a note (N) or star power (S)
         'note' : [],        # What the note is 
         'duration': []}     # tick duration of the note or star power

for note in expertsingle:
    line = note.split(' ')
    if line[2] == 'E':       # skip the lines that mark events
        continue
    else:
        notes['tick'].append(int(line[0]))
        notes['N_S'].append(line[2])
        notes['note'].append(line[3])
        notes['duration'].append(int(line[4]))

In [90]:
# Parse the 'song' section of the .chart file to get relevent chart information
song_metadata = {'Name' : '',
                 'Artist' : '',
                 'Charter' : '',
                 'Offset' : '',
                 'Resolution' : '',
                 'Genre' : '',
                 'MediaType' : '',
                 'MusicStream' : ''}

for data in song:
    line = data.split(' ')
    if line[0] in song_metadata.keys():
        song_metadata[line[0]] = line[-1]
song_metadata['Offset'] = int(song_metadata['Offset'])
song_metadata['Resolution'] = int(song_metadata['Resolution'])

{'Name': '"Exploder"', 'Artist': '"Audioslave"', 'Charter': '"Chezy"', 'Offset': 0, 'Resolution': 192, 'Genre': '"rock"', 'MediaType': '"cd"', 'MusicStream': '"song.ogg"'}


# 2. Convert ticks, resolution, and BPM to fit system requirements

Each tick must correspond to 10ms of time with BPM = 120. 

The tensors each correspond to a 10ms window of time. 

This is why it is useful to change the ticks to represent 10ms each.

## Tick Conversion Formula
We'll say each 'bin' of a particular BPM and time signature corresponds to a section, $n$, of the song with $N$ sections total.

To get the rate the ticks, $R$, we use the following formula:

$R \frac{seconds}{tick} = resolution^{-1} \frac{measures}{tick} * Time\ Signature \frac{beats}{measure} * BPM^{-1} \frac{minutes}{beat} * 60 \frac{seconds}{minute}$

$R = \frac{60 * TS}{res * BPM}$

We want to convert this tick rate to a new tick rate, $\tilde{R} = .01 \frac{seconds}{tick}$

Let $X$ be the conversion factor.

$\tilde{R} = X*R$

$X = \frac{\tilde{R}}{R} = \frac{0.01}{R}$

$X = \frac{res * BPM}{6000 * TS}$

In [142]:
# Split the song into bins corresponding to particular time signatures and BPMs

# First, assemble some lists from the preprocessing step
note_keys = list(zip(notes['tick'], notes['N_S'], 
                 notes['note'], notes['duration']))                    # (tick, 'N_S', note)
TS_events = list(zip(time_signatures['tick'], time_signatures['TS']))  # (tick, TS)
BPM_events = list(zip(BPMs['tick'], BPMs['BPM']))                      # (tick, BPM)

# Append None at the end of these lists so the loop knows where to stop
TS_events.append(None)
BPM_events.append(None)

# Loop through all the notes in the song
TS_index = 0
BPM_index = 0

cur_TS = TS_events[TS_index]                # Current time signature
cur_BPM = BPM_events[BPM_index]             # Current BPM
next_TS = None                              # Next time signature
next_BPM = None                             # Next BPM
if len(TS_events) > 1:
    next_TS = TS_events[TS_index + 1]
if len(BPM_events) > 1:
    next_BPM = BPM_events[BPM_index + 1]

# bins['TS'][0] corresponds to the time signature of bin 0
# bins['notes'] is a list of lists of notes in each bin
bins = {
    'TS' : [],              # time signature
    'BPM' : [],             # BPM
    'shift_tick' : [],      # The first tick where the TS / BPM combo starts
    'notes' : [[]]            # The notes in the bin
}

# Append the first element of each array before looping
event_index = 0     # Counts how many times either BPM or TS change
bins['TS'].append(cur_TS[1])
bins['BPM'].append(cur_BPM[1])
bins['shift_tick'].append(cur_BPM[0])
bins['notes'][event_index].append(note_keys[0])

# Initialize ticks
cur_TS_tick = cur_TS[0]
if next_TS != None:
    next_TS_tick = next_TS[0]
else:
    next_TS_tick = None
cur_BPM_tick = cur_BPM[0]
if next_BPM != None:
    next_BPM_tick = next_BPM[0]
else:
    next_BPM_tick = None

for i in range(1, len(note_keys)):
    if next_BPM_tick == None and next_TS_tick == None:  # If in the last bin
        bins['notes'][-1].append(note_keys[i])             # Add notes until there are no more to add
        continue
    
    if next_TS_tick != None:                        # If there is a time signature change in the future 
        if note_keys[i][0] >= next_TS_tick:         # If the current note is past that change                               
            if next_BPM_tick != None:                   # If there is a BPM change in the future
                if note_keys[i][0] >= next_BPM_tick:    # If the current note is past that change
                    TS_index += 1                       # Update time signature and BPM, they changed at the same time
                    cur_TS = TS_events[TS_index]
                    cur_TS_tick = cur_TS[0]
                    next_TS = TS_events[TS_index + 1]
                    next_TS_tick = next_TS[0]

                    BPM_index += 1
                    cur_BPM = BPM_events[BPM_index]
                    cur_BPM_tick = cur_BPM[0]
                    next_BPM = BPM_events[BPM_index + 1]
                    if next_BPM != None:
                        next_BPM_tick = next_BPM[0]
                    else:
                        next_BPM_tick = None

                    bins['TS'].append(cur_TS[1])
                    bins['BPM'].append(cur_BPM[1])
                    bins['shift_tick'].append(min(cur_TS[0], cur_BPM[0]))
                    bins['notes'].append([])
                    bins['notes'][-1].append(note_keys[i])
                    continue

                else:                                   # If the time signature changed but the BPM didn't
                    TS_index += 1                       # Update the time signature, but not the BPM
                    cur_TS = TS_events[TS_index]
                    cur_TS_tick = cur_TS[0]
                    next_TS = TS_events[TS_index + 1]
                    next_TS_tick = next_TS[0]

                    bins['TS'].append(cur_TS[1])
                    bins['BPM'].append(cur_BPM[1])
                    bins['shift_tick'].append(min(cur_TS[0], cur_BPM[0]))
                    bins['notes'].append([])
                    bins['notes'][-1].append(note_keys[i])
                    continue

            else:                               # If the next BPM tick = None but the note tick is past the time signature
                TS_index += 1                   # Update the time signature, but not the BPM
                cur_TS = TS_events[TS_index]
                cur_TS_tick = cur_TS[0]
                next_TS = TS_events[TS_index + 1]
                next_TS_tick = next_TS[0]

                bins['TS'].append(cur_TS[1])
                bins['BPM'].append(cur_BPM[1])
                bins['shift_tick'].append(min(cur_TS[0], cur_BPM[0]))
                bins['notes'].append([])
                bins['notes'][-1].append(note_keys[i])
                continue

        else:  # If there is a time signature change in the future but the note is not past it
            if next_BPM_tick != None:                   # If there is a BPM change in the future
                if note_keys[i][0] >= next_BPM_tick:    # If the note is past that BPM change    
                    BPM_index += 1                      # Update the BPM but not the time signature
                    cur_BPM = BPM_events[BPM_index]
                    cur_BPM_tick = cur_BPM[0]
                    next_BPM = BPM_events[BPM_index + 1]
                    if next_BPM != None:
                        next_BPM_tick = next_BPM[0]
                    else:
                        next_BPM_tick = None

                    bins['TS'].append(cur_TS[1])
                    bins['BPM'].append(cur_BPM[1])
                    bins['shift_tick'].append(min(cur_TS[0], cur_BPM[0]))
                    bins['notes'].append([])
                    bins['notes'][-1].append(note_keys[i])
                    continue

                else:  # If the time signature did not change and the BPM also did not change
                    bins['notes'][-1].append(note_keys[i])  # Add note and continue
                    continue

    #-------------------------------------------------------------------------------------------------------#
    # The second half of the ifzilla:
    # If there is not a time signature change in the future

    else:                        # If there is NOT a time signature change in the future                              
        if next_BPM_tick != None:                   # If there is a BPM change in the future
            if note_keys[i][0] >= next_BPM_tick:    # If the current note is past that change
                BPM_index += 1                      # Update the BPM
                cur_BPM = BPM_events[BPM_index]
                cur_BPM_tick = cur_BPM[0]
                next_BPM = BPM_events[BPM_index + 1]
                if next_BPM != None:
                    next_BPM_tick = next_BPM[0]
                else:
                    next_BPM_tick = None

                bins['TS'].append(cur_TS[1])
                bins['BPM'].append(cur_BPM[1])
                bins['shift_tick'].append(min(cur_TS[0], cur_BPM[0]))
                bins['notes'].append([])
                bins['notes'][-1].append(note_keys[i])
                continue

            else:  # If the current note is not past the BPM change
                bins['notes'][-1].append(note_keys[i])  # Add note and continue
                continue

        else:                               # If the next BPM tick = None and the next TS tick = None
            print('Why am I here?')         # Then the if statement at the beginning of this thing should have fired off

    
    # if next_TS:
    #     next_TS_tick = next_TS[0]
    # else:
    #     next_TS_tick = None

    # if next_BPM:
    #     next_BPM_tick = next_BPM[0]
    # else:
    #     next_BPM_tick = None


{'TS': [4], 'BPM': [95500], 'shift_tick': [0], 'notes': [[(768, 'N', '3', 0)]]}
[(768, 'N', '3', 0), (864, 'N', '3', 0), (960, 'N', '3', 0), (1056, 'N', '3', 0), (1152, 'N', '3', 0), (1248, 'N', '3', 0), (1344, 'N', '3', 0), (1440, 'N', '3', 0), (1536, 'N', '3', 0), (1632, 'N', '3', 0)]


In [147]:
print(len(bins['shift_tick']))

256
